# Gradio with OpenAI API (via OpenRouter)

<a target="_blank" href="https://colab.research.google.com/github/simonguest/CS-394/blob/main/src/02/notebooks/gradio.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/simonguest/CS-394/raw/refs/heads/main/src/02/notebooks/gradio.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

## Install required packages

In [1]:
!uv add gradio==5.49.1 openai

Resolved 190 packages in 1ms
Installed 49 packages in 208ms                              
 + aiofiles==24.1.0
 + annotated-doc==0.0.4
 + annotated-types==0.7.0
 + audioop-lts==0.2.2
 + brotli==1.2.0
 + click==8.3.1
 + contourpy==1.3.3
 + cycler==0.12.1
 + distro==1.9.0
 + fastapi==0.128.0
 + ffmpy==1.0.0
 + filelock==3.20.1
 + fonttools==4.61.1
 + fsspec==2025.12.0
 + gradio==5.49.1
 + gradio-client==1.13.3
 + groovy==0.1.2
 + hf-xet==1.2.0
 + huggingface-hub==0.36.0
 + jiter==0.12.0
 + kiwisolver==1.4.9
 + markdown-it-py==4.0.0
 + matplotlib==3.10.8
 + mdurl==0.1.2
 + numpy==2.4.0
 + openai==2.14.0
 + orjson==3.11.5
 + pandas==2.3.3
 + pillow==11.3.0
 + pydantic==2.11.10
 + pydantic-core==2.33.2
 + pydub==0.25.1
 + pyparsing==3.3.1
 + python-dotenv==1.2.1
 + python-multipart==0.0.21
 + pytz==2025.2
 + rich==14.2.0
 + ruff==0.14.12
 + safehttpx==0.1.7
 + semantic-version==2.10.0
 + shellingham==1.5.4
 + sniffio==1.3.1
 + starlette==0.50.0
 + tomlkit==0.13.3
 + tqdm==4.67.1
 + typer==0.

## Set the OpenRouter API Key from Colab Secrets

In [2]:
from google.colab import userdata
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')

ModuleNotFoundError: No module named 'google'

## (Or grab the OpenRouter API key if running locally)

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")

## Initialize the OpenAI client

In [4]:
import openai

# Initialize OpenAI client
client = openai.OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=OPENROUTER_API_KEY,
)

## Example 1: Basic Gradio interface

In [5]:
import gradio as gr

def image_classifier(inp):
    return {'cat': 0.3, 'dog': 0.7}

demo = gr.Interface(fn=image_classifier, inputs="image", outputs="label")
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Example 2: Basic chat interface with conversation history

In [6]:
import gradio as gr

def chat_with_history(message, history):
    # Add current message
    messages = history + [{"role": "user", "content": message}]
    
    # Get response from API
    response = client.chat.completions.create(
        model='openai/gpt-5.2-chat',
        messages=messages,
    )
    
    return response.choices[0].message.content

# Create a chat interface
demo = gr.ChatInterface(
    fn=chat_with_history,
    title="Basic Chat with Conversation History"
)

demo.launch()

/Users/simon/Dev/CS-394/.venv/lib/python3.13/site-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## Example 3: Streaming chat interface

In [7]:
def chat_with_streaming(message, history):
    messages = history + [{"role": "user", "content": message}]
    
    # Stream the response
    stream = client.chat.completions.create(
        model='openai/gpt-5.2-chat',
        messages=messages,
        stream=True,
    )
    
    response_text = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            token = chunk.choices[0].delta.content
            response_text += token
            yield response_text

# Create streaming chat interface
demo = gr.ChatInterface(
    fn=chat_with_streaming,
    title="AI Chat with Streaming",
)

demo.launch()

/Users/simon/Dev/CS-394/.venv/lib/python3.13/site-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
